참고 : https://hwanny-yy.tistory.com/3

In [1]:
import os
from PIL import Image

import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms

from torch.utils.data import DataLoader

from torchvision.models.detection import maskrcnn_resnet50_fpn

import numpy as np
import cv2
import random


C:\Users\ay011\anaconda3\envs\dsl\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: [WinError 126] 지정된 모듈을 찾을 수 없습니다
  warn(f"Failed to load image Python extension: {e}")


# Dataset

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

- data 폴더 내 사진들이 모여있는 구조
- size를 일치시켜야 하는 문제 존재

In [3]:
class ImgDataset(Dataset):
    def __init__(self, transforms_=None):
        self.transform = transforms.Compose(transforms_)
        files = []
        file_list = os.listdir('data/')
        for file in file_list:
            # 이 때 Img의 길이를 일치시켜야 하는 문제 존재
            img = Image.open(f'data/{file}').resize((1024, 1024), Image.LANCZOS)
            files.append(self.transform(img))

        self.files = files

    def __getitem__(self, index):
        return self.files[index].to(device)

    def __len__(self):
        return len(self.files)

# Util

- pretrained 모델 그대로 사용해서 생기는 문제로 생각
> 의류 데이터로 tuning 필요

In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# COCO dataset
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

## Tensor > Image

In [5]:
def tensor2img(tensor):
    tensor = 127.5 * (tensor[0].data.cpu().float().numpy() + 1.0)
    img = tensor.astype(np.uint8)
    img = np.transpose(img, (1, 2, 0))
    return img

## Mask 시각화

In [6]:
def apply_mask(image, mask, labels, boxes, file_name):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    alpha = 1
    beta = 0.6  # transparency for the segmentation map
    gamma = 0  # scalar added to each sum
    COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))
    _, _, w, h = mask.shape
    segmentation_map = np.zeros((w, h, 3), np.uint8)

    for n in range(mask.shape[0]):
        if labels[n] == 0:
            continue
        else:
            color = COLORS[random.randrange(0, len(COLORS))]
            segmentation_map[:, :, 0] = np.where(mask[n] > 0.5, COLORS[labels[n]][0], 0)
            segmentation_map[:, :, 1] = np.where(mask[n] > 0.5, COLORS[labels[n]][1], 0)
            segmentation_map[:, :, 2] = np.where(mask[n] > 0.5, COLORS[labels[n]][2], 0)
            image = cv2.addWeighted(image, alpha, segmentation_map, beta, gamma, dtype=cv2.CV_8U)

        # draw the bounding boxes around the objects
        cv2.rectangle(image, boxes[n][0], boxes[n][1], color=color, thickness=2)

        print(class_names[labels[n]])
        # put the label text above the objects
        cv2.putText(image, class_names[labels[n]], (boxes[n][0][0], boxes[n][0][1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color,
                    thickness=2, lineType=cv2.LINE_AA)
    # image save
    cv2.imwrite(f'save/{file_name}.png', image)

# Process

In [7]:
# from dataset import ImgDataset
# from utils import tensor2img, apply_mask



device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

batch_size = 1
score_threshold = 0.965

## 데이터 셋 로드

In [8]:
transform = [transforms.ToTensor()]
dataset = ImgDataset(transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

## 결과 확인

In [9]:
model = maskrcnn_resnet50_fpn(pretrained=True).to(device)

model.eval()
for idx, data in enumerate(dataloader):

    result = model(data)

    image = tensor2img(data)
    scores = list(result[0]['scores'].detach().cpu().numpy())
    thresholded_preds_inidices = [scores.index(i) for i in scores if i > score_threshold]
    thresholded_preds_count = len(thresholded_preds_inidices)
    mask = result[0]['masks']
    mask = mask[:thresholded_preds_count]
    labels = result[0]['labels']
    boxes = [[(int(i[0]), int(i[1])), (int(i[2]), int(i[3]))] for i in result[0]['boxes']]
    boxes = boxes[:thresholded_preds_count]

    mask = mask.data.float().cpu().numpy()

    apply_mask(image, mask, labels, boxes, idx)

C:\Users\ay011\anaconda3\envs\dsl\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


RuntimeError: Couldn't load custom C++ ops. This can happen if your PyTorch and torchvision versions are incompatible, or if you had errors while compiling torchvision from source. For further information on the compatible versions, check https://github.com/pytorch/vision#installation for the compatibility matrix. Please check your PyTorch version with torch.__version__ and your torchvision version with torchvision.__version__ and verify if they are compatible, and if not please reinstall torchvision so that it matches your PyTorch install.